In [1]:
from tqdm import tqdm
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, accuracy_score
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import optuna
import lightgbm as lgb

In [2]:
df = pd.read_csv('data.csv')

In [3]:
RANDOM_STATE = 42

TRAIN_SIZE = 0.6
VAL_SIZE = 0.15
TEST_SIZE = 0.25

TARGET = 'attr_value_name'

In [86]:
def calc_metrics(
        model: LogisticRegression,
        vect: CountVectorizer | TfidfVectorizer,
        sample_part: str = 'test',
        multi_class: str = 'ovr',
        descr_column: str = 'description'
):
    """Функция для расчета метрики
    """
    descr_vect = vect.transform(df.loc[df['sample_part'] == sample_part, descr_column])
    proba = model.predict_proba(descr_vect)
    prediction = model.predict(descr_vect)
    target = df.loc[df['sample_part'] == sample_part, TARGET]

    auc_macro = roc_auc_score(
        y_score=proba,
        y_true=target,
        average='macro',
        multi_class=multi_class
    )
    auc_micro = roc_auc_score(
        y_score=proba,
        y_true=target,
        average='micro',
        multi_class=multi_class
    )

    accuracy = accuracy_score(
        y_pred=prediction,
        y_true=target
    )

    print(f'Metrics on {sample_part}:\n'
          f'\tAUC (macro): {auc_macro:.3f}\n'
          f'\tAUC (micro): {auc_micro:.3f}\n'
          f'\tAccuracy: {accuracy:.3f}\n')

Разбиение выборки

In [9]:
df_splitting = pd.read_csv('splitting_sample.csv')
df = df.merge(df_splitting, on='item_id', how='left')

Подгружаем разметку базелином

In [10]:
df_baseline = pd.read_csv('baseline_prediction.csv')
df = df.merge(df_baseline, on='item_id', how='left')

# BoW (без предобработки теста)

In [25]:
russian_stopwords = stopwords.words("russian")
russian_stopwords[:10]

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со']

In [26]:
bow_vect = CountVectorizer(
    lowercase=True,
    preprocessor=None,
    tokenizer=None,
    stop_words=russian_stopwords,
    token_pattern='(?u)\\b\\w\\w+\\b',
    ngram_range=(1, 3),
    analyzer='word',
    max_df=1.0,
    min_df=100,
    max_features=None
)

bow_vect.fit(df.loc[df['sample_part'] == 'train', 'description'])

CountVectorizer(min_df=100, ngram_range=(1, 3),
                stop_words=['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с',
                            'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его',
                            'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы',
                            'по', 'только', 'ее', 'мне', ...])

In [31]:
len(bow_vect.vocabulary_)

17449

Обучаем логрег

In [37]:
lr_bow = LogisticRegression(
    random_state=RANDOM_STATE,
    max_iter=500,
    n_jobs=-1,
    multi_class='ovr'
)

lr_bow.fit(
    X=bow_vect.transform(df.loc[df['sample_part'] == 'train', 'description']),
    y=df.loc[df['sample_part'] == 'train', TARGET]
)

LogisticRegression(max_iter=500, multi_class='ovr', n_jobs=-1, random_state=42)

In [48]:
calc_metrics(
    model=lr_bow,
    vect=bow_vect,
    sample_part='test'
)

Metrics on test:
	AUC (macro): 0.818
	AUC (micro): 0.856
	Accuracy: 0.632



Неплохо.. Во всяком случае базелин побит

# TF-IDF (без предобработки текста)

In [57]:
tfidf_vect = TfidfVectorizer(
    lowercase=True,
    preprocessor=None,
    tokenizer=None,
    stop_words=russian_stopwords,
    token_pattern='(?u)\\b\\w\\w+\\b',
    ngram_range=(1, 3),
    analyzer='word',
    max_df=1.0,
    min_df=100,
    max_features=None,
    norm='l2'
)

tfidf_vect.fit(df.loc[df['sample_part'] == 'train', 'description'])
len(tfidf_vect.vocabulary_)

17449

Логрег

In [58]:
lr_tfidf = LogisticRegression(
    random_state=RANDOM_STATE,
    max_iter=500,
    n_jobs=-1,
    multi_class='ovr'
)

lr_tfidf.fit(
    X=tfidf_vect.transform(df.loc[df['sample_part'] == 'train', 'description']),
    y=df.loc[df['sample_part'] == 'train', TARGET]
)

LogisticRegression(max_iter=500, multi_class='ovr', n_jobs=-1, random_state=42)

In [59]:
calc_metrics(
    model=lr_tfidf,
    vect=tfidf_vect,
    sample_part='test'
)

Metrics on test:
	AUC (macro): 0.855
	AUC (micro): 0.883
	Accuracy: 0.668



In [120]:
# сохраняем предсказания
df['tfidf_lr_predict'] = lr_tfidf.predict(tfidf_vect.transform(df['description']))
df['tfidf_lr_proba'] = lr_tfidf.predict_proba(tfidf_vect.transform(df['description'])).max(axis=1)

Есть улучшения

# TF-IDF (со стеммингом)

In [82]:
tfidf_stem_vect = TfidfVectorizer(
    lowercase=True,
    preprocessor=None,
    tokenizer=None,
    stop_words=russian_stopwords,
    token_pattern='(?u)\\b\\w\\w+\\b',
    ngram_range=(1, 3),
    analyzer='word',
    max_df=1.0,
    min_df=100,
    max_features=None,
    norm='l2'
)

ru_stemmer = SnowballStemmer('russian')
tokenizer = tfidf_stem_vect.build_tokenizer()
text_tokens = [tokenizer(text) for text in tqdm(df['description'].values)]
df['description_stem'] = [
    ' '.join([ru_stemmer.stem(word) for word in text]) for text in tqdm(text_tokens)
    ]

tfidf_stem_vect.fit(df.loc[df['sample_part'] == 'train', 'description_stem'])
len(tfidf_stem_vect.vocabulary_)

100%|██████████| 199344/199344 [06:07<00:00, 541.70it/s]


17852

In [83]:
lr_tfidf_stem = LogisticRegression(
    random_state=RANDOM_STATE,
    max_iter=500,
    n_jobs=-1,
    multi_class='ovr'
)

lr_tfidf_stem.fit(
    X=tfidf_stem_vect.transform(df.loc[df['sample_part'] == 'train', 'description_stem']),
    y=df.loc[df['sample_part'] == 'train', TARGET]
)

LogisticRegression(max_iter=500, multi_class='ovr', n_jobs=-1, random_state=42)

In [87]:
calc_metrics(
    model=lr_tfidf_stem,
    vect=tfidf_stem_vect,
    sample_part='test',
    descr_column='description_stem'
)

Metrics on test:
	AUC (macro): 0.856
	AUC (micro): 0.883
	Accuracy: 0.669



Значительного увеличения качества по сравнению с обычным TF-IDF нет

# TFIDF (без предобработки текста) + бустинг

In [89]:
tfidf_vect = TfidfVectorizer(
    lowercase=True,
    preprocessor=None,
    tokenizer=None,
    stop_words=russian_stopwords,
    token_pattern='(?u)\\b\\w\\w+\\b',
    ngram_range=(1, 3),
    analyzer='word',
    max_df=1.0,
    min_df=100,
    max_features=5000, # устанавливаем ограничение (иначе бустинг может не переварить много фичей)
    norm='l2'
)

tfidf_vect.fit(df.loc[df['sample_part'] == 'train', 'description'])
len(tfidf_vect.vocabulary_)

5000

## Без тюнинга гиперпараметров

In [102]:
clf_boost = lgb.LGBMClassifier(
    random_state=RANDOM_STATE,
    n_jobs=-1,
    verbose=-1
)
clf_boost.fit(
    X=tfidf_vect.transform(df.loc[df['sample_part'] == 'train', 'description']),
    y=df.loc[df['sample_part'] == 'train', TARGET]
)

LGBMClassifier(n_jobs=-1, random_state=42, verbose=-1)

In [103]:
calc_metrics(
    model=clf_boost,
    vect=tfidf_vect,
    sample_part='test',
    descr_column='description'
)

Metrics on test:
	AUC (macro): 0.854
	AUC (micro): 0.882
	Accuracy: 0.663



Результаты чуть хуже

## После тюнинга

In [113]:
BASE_PARAMS = {
    'random_state': RANDOM_STATE,
    'n_jobs': -1,
    'verbose': -1
}
TRAIN_SAMPLES = np.sum(df['sample_part'] == 'train')

TRAIN_DATA_TFIDF = tfidf_vect.transform(df.loc[df['sample_part'] == 'train', 'description'])
VAL_DATA_TFIDF = tfidf_vect.transform(df.loc[df['sample_part'] == 'val', 'description'])
TRAIN_TARGET_DATA = df.loc[df['sample_part'] == 'train', TARGET]
VAL_TARGET_DATA = df.loc[df['sample_part'] == 'val', TARGET]

def objective(trial):
    params = {**BASE_PARAMS}
    params['classifier'] = trial.suggest_categorical('classifier', ['gbdt', 'rf'])
    params['learning_rate'] = trial.suggest_float('learning_rate', 1e-3, 5e-1, log=True)
    params['n_estimators'] = trial.suggest_int('n_estimators', 100, 500)
    params['min_child_samples'] = trial.suggest_int('min_child_samples', 0.01 * TRAIN_SAMPLES, 0.2 * TRAIN_SAMPLES)

    clf = lgb.LGBMClassifier(**params)
    clf.fit(
        X=TRAIN_DATA_TFIDF,
        y=TRAIN_TARGET_DATA,
        eval_set=(
            (VAL_DATA_TFIDF, VAL_TARGET_DATA)
        ),
        eval_metric='logloss',
        callbacks=[lgb.early_stopping(30)]
    )

    return roc_auc_score(
        y_score=clf.predict_proba(VAL_DATA_TFIDF),
        y_true=VAL_TARGET_DATA,
        average='macro',
        multi_class='ovr'
    )

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2024-05-11 11:56:46,883] A new study created in memory with name: no-name-eaa5478d-23e1-410b-addd-2095dbc260f6


Training until validation scores don't improve for 30 rounds


[I 2024-05-11 11:56:48,900] Trial 0 finished with value: 0.6449324210628906 and parameters: {'classifier': 'rf', 'learning_rate': 0.0063859850056025395, 'n_estimators': 126, 'min_child_samples': 14508}. Best is trial 0 with value: 0.6449324210628906.


Did not meet early stopping. Best iteration is:
[126]	valid_0's multi_logloss: 1.19679
Training until validation scores don't improve for 30 rounds
Did not meet early stopping. Best iteration is:
[355]	valid_0's multi_logloss: 1.16419


[I 2024-05-11 11:56:53,325] Trial 1 finished with value: 0.6612650295668818 and parameters: {'classifier': 'rf', 'learning_rate': 0.017121831199215787, 'n_estimators': 355, 'min_child_samples': 18637}. Best is trial 1 with value: 0.6612650295668818.


...

[I 2024-05-11 13:01:23,811] Trial 97 finished with value: 0.8172905366690848 and parameters: {'classifier': 'rf', 'learning_rate': 0.07388042279680923, 'n_estimators': 414, 'min_child_samples': 3719}. Best is trial 21 with value: 0.8472560442891142.


Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[309]	valid_0's multi_logloss: 0.855254


[I 2024-05-11 13:02:24,373] Trial 98 finished with value: 0.8404061399662606 and parameters: {'classifier': 'rf', 'learning_rate': 0.10368419997326811, 'n_estimators': 491, 'min_child_samples': 1759}. Best is trial 21 with value: 0.8472560442891142.


Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[249]	valid_0's multi_logloss: 0.892244


[I 2024-05-11 13:02:53,532] Trial 99 finished with value: 0.8257033301124913 and parameters: {'classifier': 'rf', 'learning_rate': 0.20519331886092454, 'n_estimators': 426, 'min_child_samples': 2815}. Best is trial 21 with value: 0.8472560442891142.


In [116]:
clf_tuned = lgb.LGBMClassifier(**BASE_PARAMS, **study.best_params)
clf_tuned.fit(
    X=TRAIN_DATA_TFIDF,
    y=TRAIN_TARGET_DATA,
    eval_set=(
        (VAL_DATA_TFIDF, VAL_TARGET_DATA)
    ),
    eval_metric='logloss',
    callbacks=[lgb.early_stopping(30)]
)

Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[366]	valid_0's multi_logloss: 0.836708


LGBMClassifier(classifier='rf', learning_rate=0.0845251495668393,
               min_child_samples=1198, n_estimators=436, n_jobs=-1,
               random_state=42, verbose=-1)

In [117]:
calc_metrics(
    model=clf_tuned,
    vect=tfidf_vect,
    sample_part='test',
    descr_column='description'
)

Metrics on test:
	AUC (macro): 0.846
	AUC (micro): 0.876
	Accuracy: 0.652



**Итог**: оставим TF-IDF без предобработки текста + логистическая регрессия.

Этот подход оптимален с точки зрения хорошего качества + низкой цены инференса

In [123]:
df[['item_id', 'tfidf_lr_predict', 'tfidf_lr_proba']].to_csv('tfidf_lr_prediction.csv', index=False)